## A propotype for generating XIOS file_def entries from CMIP6 Data Request API for a given year of an experiment, and a modeling realm

### First version : Stéphane Sénési - Feb 2016

#### File_def entries will refer to XIOS 'fields' which names are CMOR variable names : we assume that some upstream field_def entries have managed to match model diagnostics to CMOR variables, both for name and units. 

#### Current shortcomings  : a lot, including that : i) a DR 'realm'is assumed to match an XIOS 'context', ii) split_freq should depend on field dimensions, iii) not all global attributes are set , iv) must clarify the variables name choice ....

### 1- Modeling center settings - should be read from a config file

In [1]:
institute_id="CNRM"
institute="Centre National de Recherches Météorologiques"
model="CNRM CM6"
model_id="CNRM-CM6"
model2="" # Offline model id - must include '--' prefix if not empty . TBD : really handle it 
#contact="cmip6.data@meteo.fr"   # Deprecated
references="..."

# We account for the list of MIPS in which the lab takes part.
# Note : a MIPs set limited to {'C4MIP'} leads to a number of tables and variables which is manageable for eye inspection
our_mips={'C4MIP'}  

# We account for a list of variables which the lab does not want to produce , 
# excluded_vars_file="../../cnrm/non_published_variables"

# Max variable priority level to be output
max_priority=3

### 2- Data defining the project, and experiment - to be set/translated by the simulation workflow from its own experiment metadata container

In [2]:
project="CMIP6"
project_id="CMIP6"
activity_label="CMIP6" # MIP specifying the experiment. For historical, it is CMIP6 itself
experiment_id="historical"
experiment="Historical ..." # Should be found in DR from exp_id
realization_index=1
initialization_index=1
physics_index=1
forcing_index=1
grid_index=0
sub_experiment_index="" # Must begin with '-E' if not empty
start_date="" # Must begin with '-' if not empty
parent_experiment_id="piControl"
parent_experiment_ripfg="r1i0p0f0g0"
# forcing="..."  # Deprecated
history="..."
ensemble_member="r%di%dp%df%d"%(realization_index, initialization_index, physics_index, forcing_index)
title="%s%s model output prepared for CMIP6 %s%s"%(model_id,model2,experiment_id,sub_experiment_index)

comment="..."  #Specific to the experiment and possibly to the table - should then be moved eslewhere

#Still missing : 
# convention, (start_date), (lead_time), creation_date, 
# branch_time_in_parent, branch_method, branch_time_units_in_parent, branch_time_in_child, 
# further_info_url, terms_of_use_tracking_id

### 3- Which year and which realm are we currently processing (should be arguments for a function)

In [3]:
year=1850
# We could conisder processing all realms at once. 
the_realm='land'

### 4- Read the Data Request 

In [4]:
from dreq import *
dq = loadDreq()

In [5]:
#for m in dq.coll['exptgroup'].items : print m.label,

In [6]:
allMIPS=[]
for m in dq.coll['mip'].items : allMIPS.append(m.label)
for m in allMIPS : print m+" ",

CMIP6  AerChemMIP  C4MIP  CFMIP  DAMIP  DCPP  FAFMIP  GeoMIP  GMMIP  HighResMIP  ISMIP6  LS3MIP  LUMIP  OMIP  PDRMIP  PMIP  RFMIP  ScenarioMIP  SolarMIP  VolMIP  CORDEX  DynVar  SIMIP  VIACSAB  SPECS  CCMI  CMIP5  DECK 


In [7]:
for e in dq.coll['experiment'].items : 
    if e.label==experiment_id :
        #e.__info__()
        pass

### 5- Use class dreqQuery and method getRequestLinks to list requestLinks for our set of MIPs

In [8]:
from scope import dreqQuery
sc = dreqQuery(dq=dq, tierMax=3)
miprls=sc.getRequestLinkByMip(our_mips)
# Look at one requestLink
rl0=miprls[0]
print rl0
print rl0.title, rl0.label, rl0.tab,rl0.ref
#miprls

Item <3.3 Request link: linking a set of variables and a set of experiments>: [__unset__] Omon
Omon __unset__ Omon CMIP5Rev


### 6- A function to filter RequestLinks by the current experiment and the year currently processed

In [9]:
def RequestItem_applies_for_exp_and_year(dq,ri,experiment,year,debug=False):
    """ Returns True if requestItem ri is relevant for a given experiment and year """
    # Acces experiment or experiment group for the RequestItem
    if (debug) : print "Checking ","% 15s"%ri.label,
    item_exp=dq.inx.uid[ri.esid]
    relevant=False
    exps=dq.coll['experiment'].items
    # esid can link to an experiment or an experiment group
    if item_exp._h.label== 'experiment' :
        if (debug) : print "%20s"%"Simple Expt case", item_exp.label,
        if item_exp.label==experiment : 
            if (debug) : print " OK",
            relevant=True
    elif item_exp._h.label== 'exptgroup' :
        if (debug)  : print "%20s"%"Expt Group case ",item_exp.label,
        group_id=ri.esid
        for e in exps :
            if 'gid' in dir(e) and e.gid == group_id and e.label==experiment : 
                if (debug) :  print " OK for experiment",
                relevant=True
    elif item_exp._h.label== 'mip' :
        mip_id=ri.esid
        if (debug)  : print "%20s"%"Mip case ",dq.inx.uid[mip_id].label,
        for e in exps :
            if 'mip' in dir(e) and e.mip == mip_id :
                if (debug) :  print e.label,",",
                if e.label==experiment : 
                    if (debug) :  print " OK for experiment",
                    relevant=True
    else :
        # TBD !! : understand what is happening in that case 
        if (debug)  : print "%20s"%'Other case , label=%s|'%item_exp._h.label,
    if relevant :
        if 'tslice' in ri.__dict__ and ri.tslice != '__unset__' :
            timeslice=dq.inx.uid[ri.tslice]
            if (debug) : print "OK for the year"
            return year >= timeslice.start and year<=timeslice.end
        else :
            # TBD : !! test once timeSlices will be set in DR
            if (debug)  : print "tslice not set -> OK for the year"
            return True
    if (debug)  : print "NOK"
    return False

### 7- Let us apply the filter

In [10]:
filtered_rls=[]
for rl in miprls :
    # Access all requesItems ids which refer to this RequestLink
    ri_ids=dq.inx.iref_by_sect[rl.uid].a['requestItem']
    for ri_id in ri_ids :
        ri=dq.inx.uid[ri_id]
        #print "Checking requestItem ",ri.label
        if RequestItem_applies_for_exp_and_year(dq,ri,experiment_id, year,False) :
            #print "% 25s"%ri.label," applies "
            filtered_rls.append(rl)

print "Number of Request Links which apply to",experiment_id,"for MIPS", list(our_mips), "and year",year,"is:",len(filtered_rls)

Number of Request Links which apply to historical for MIPS ['C4MIP'] and year 1850 is: 15


### 8- Use dreqQuery method varsByRql to get the list of CMOR variables for these RequestLinks (using max_priority)

In [11]:
miprl_ids=[ rl.uid for rl in filtered_rls ]
miprl_vars=sc.varsByRql(miprl_ids, pmax=max_priority)
print 'Numer of CMOR variables for these requestLinks is :',len(miprl_vars)
miprl_vars_list=list(miprl_vars)

Numer of CMOR variables for these requestLinks is : 735


### 9- Have a look at one CMOR variable

In [12]:
v0=dq.inx.uid[miprl_vars_list[0]]
print v0.label, v0.modeling_realm, v0.frequency, v0.mipTable, v0.mipTableSection,v0.type
#dir(v0)

froc ocnBgchem mon Omon __unset__ real


### 10- Group CMOR variables per modeling realm, or per XIOS contex (in a dict of lists)

In [13]:
cmvs_per_realm=dict()
for cmv_uid in miprl_vars_list :
    cmv=dq.inx.uid[cmv_uid]
    # Here, we should use the XIOS context where the variable 
    # is defined, rather than the realm provided by the DR
    # This assumes to access some dict (config file ?) describing these exceptions
    if cmv.modeling_realm not in cmvs_per_realm :
        cmvs_per_realm[cmv.modeling_realm]=[]
    cmvs_per_realm[cmv.modeling_realm].append(cmv_uid)
print cmvs_per_realm.keys()

['seaIce', 'land', 'atmos atmosChem', 'ocean seaIce', 'landIce land', 'ocean', 'atmos', 'land landIce', 'ocnBgchem']


### 11- Read the list of the variables that the laboratory does not want to produce

In [17]:
excluded_vars=[]
use_lab_list_for_excluded_variables=False
if use_lab_list_for_excluded_variables :
    with open(excluded_vars_file) as fi :
        for li in fi.readlines(): 
            # Assume a very simple structure for the file : one 
            # var per line, with possibly comment lines
            if li[0] != '#' :  excluded_vars.append(li)

### 12- Filter CMOR variables on the requested realm (or context) and with this list, and group them per table

In [18]:
cmvs_pertable=dict()
for cmv_uid in cmvs_per_realm[the_realm] :
    cmv=dq.inx.uid[cmv_uid]
    if cmv.mipTable not in cmvs_pertable :
        cmvs_pertable[cmv.mipTable]=[]
    if cmv.label not in excluded_vars :
        cmvs_pertable[cmv.mipTable].append(cmv)
print cmvs_pertable.keys()

['emMon', 'em3hr', 'emDay']


### 13- Print CMOR variable names, grouped perTable

In [19]:
for table in cmvs_pertable :    
    print "% 15s"%table," ",len(cmvs_pertable[table])," ----> ",
    for cmv in cmvs_pertable[table] : 
        print cmv.label,
    print

          emMon   136  ---->  burntFractionAll fFireAll netAtmosLandC13Flux vegHeightTree rh-c14 fg13co2 sf6 c13Litter nStem cropFracC3 treeFracBdlEvg nOther cRoot fProductDecomp fHarvestToProduct vegHeight cropFracC4 c14Veg cProduct nRoot fVegSoil wetlandFrac shrubFrac cLeaf fVegFire nMineral raRoot mrso cStem fNVegSoil cVeg mrsol tran vegFrac grassFracC4 fNVegLitter rhSoil c14Land vegHeightShrub fAnthDisturb gpp-c14 fNLandToOcean nLand fNgas mrsll fNgasFire nLitterCwd grassFracC3 c13Soil c13Land c14Litter residualFrac treeFracNdlEvg fVegLitterSenescence wetlandCH4cons c13Veg fNOx evspsblveg fVegLitter fBNF raStem treeFracNdlDcd nMineralNH4 vegHeightGrass wetlandCH4prod fHarvestToAtmos fFireNat fgcfc12 rh-c13 evspsblsoi fDeforestToProduct cLand ra-c14 nLitterFine fDeforestToAtmos fNloss grassFrac cSoil1m fNup fgsf6 dissoc c14Soil cOther cropFrac mrros dissi14c waterDpth npp netAtmosLandCO2Flux vegHeightCrop lai cLitterSurf cLitterSubSurf nLitter fCLandToOcean cLitterCwd fN2O mrlso cLi

### 14- Generate an XIOS file_def entry for each table

In [20]:
def file_def(dic,table) :
    """ From a dictionnary dic which keys are tables and entries are lists of CMOR variables,
    generate an XIOS file_def entry . Uses a global dq object"""
    # TBD : identify the frequency from the table and translate it to XIOS syntax
    freq="1d"
    # WIP Draft on filenames, feb 2016
    # <variable>_<table>_<model>[--<model2>]_<experiment>[-{startdate}][-E{sub-experiment index}]_{ensemble_member}[_{time_range}][_{processing}].nc, 
    time_range="$TIME_RANGE_PATTERN_FOR_XIOS$"
    processing="_g0" # Must begin with '_' if not empty
    # TBD : logic for computing file level split_frequency from set of variables and their rank ?
    out.write('<file name="%s_%s%s_%s%s%s_%s%s%s" '%\
               (table,model_id,model2,experiment,start_date,sub_experiment_index,ensemble_member,time_range,processing))
    out.write(' freq_output="%s" append="true" split_freq="10y" timeseries="exclusive" >\n'%freq)
    out.write('  <variable name="project_id" type="string" > %s/%s </variable>\n'%(project,activity_label))
    out.write('  <variable name="institute_id"  type="string" > %s </variable>\n'%institute_id)
    out.write('  <variable name="institute"     type="string" > %s </variable>\n'%institute)
    out.write('  <variable name="model_id"      type="string" > %s </variable>\n'%model_id)
    out.write('  <variable name="model"         type="string" > %s </variable>\n'%model)
    out.write('  <variable name="experiment_id" type="string" > %s </variable>\n'%experiment_id)
    out.write('  <variable name="experiment"    type="string" > %s </variable>\n'%experiment)
    out.write('  <variable name="frequency"     type="string" > %s </variable>\n'%freq)
    out.write('  <variable name="realization_index"    type="string" > %s </variable>\n'%realization_index)
    out.write('  <variable name="physics_index"        type="string" > %s </variable>\n'%physics_index)
    out.write('  <variable name="initialization_index" type="string" > %s </variable>\n'%initialization_index)
    out.write('  <variable name="forcing_index"        type="string" > %s </variable>\n'%forcing_index)
    out.write('  <variable name="grid_index"           type="string" > %s </variable>\n'%grid_index)
    out.write('  <variable name="parent_experiment_id" type="string" > %s </variable>\n'%parent_experiment_id)
    out.write('  <variable name="parent_experiment_ripfg" type="string" > %s </variable>\n'%parent_experiment_ripfg)
    out.write('  <variable name="table_id"       type="string" > %s </variable>\n'%table)
    out.write('  <variable name="comment"        type="string" > %s </variable>\n'%comment)
    #out.write('  <variable name="contact"        type="string" > %s </variable>\n'%contact)
    #out.write('  <variable name="forcing"        type="string" > %s </variable>\n'%forcing)
    out.write('  <variable name="history"        type="string" > %s </variable>\n'%history)
    out.write('  <variable name="references"     type="string" > %s </variable>\n'%references)
    out.write('  <variable name="title"          type="string" > %s </variable>\n'%title)
    if sub_experiment_index is not None and sub_experiment_index != '' :
        out.write('    <variable name="sub_experiment_index"           type="string" > %s </variable>\n'%sub_experiment_index)
    #Still missing : convention, start_date, lead_time, creation_date, branch_time_in_parent, 
    #branch_method, branch_time_units_in_parent, branch_time_in_child, further_info_url, terms_of_use_tracking_id
    
    for cmv in cmvs_pertable[table] : 
        # TBD : identify the cell_method wrt to time and translate it to XIOS time_operation
        # TBD : logic for computing ts_split_frequency from rank
        mipvar=dq.inx.uid[cmv.vid]
        stdname=dq.inx.uid[mipvar.sn]
        if stdname._h.label != 'standardname':
            # Assume that no CF standard is defined for this MIP variable -> 
            # use MIP variable entries for name and units
            slabel=cmv.label
            sunits=mipvar.units
            sdescription=mipvar.description
        else :
            slabel=stdname.label
            sunits=stdname.units
            sdescription=stdname.description[0:30]
        structure=dq.inx.uid[cmv.stid]
        op="average" # Should be deduced from 'time:' part of cell_method
        out.write('  <field field_ref="CF_%s" operation="%s" ts_enabled="true" ts_split_freq="10y">\n'%(slabel,op))
        out.write('     <variable name="standard_name" type="string" > %s </variable>\n'%slabel)
        out.write('     <variable name="long_name"     type="string" > %s </variable>\n'%mipvar.title)
        out.write('     <variable name="description"   type="string" > %s ...  </variable>\n'%sdescription)
        if cmv.positive != "None" and cmv.positive != "" :
            out.write('     <variable name="positive"      type="string" > %s </variable>\n'%cmv.positive)
        out.write('     <variable name="cell_methods"  type="string" > %s </variable>\n'%structure.cell_methods)
        out.write('     <variable name="cell_measures" type="string" > %s </variable>\n'%structure.cell_measures)
        out.write('     </field>\n')
    out.write('<file/>\n')

with open("%s.xml"%the_realm,"w") as out :
    #for table in cmvs_pertable :    
    for table in [ "em3hr" ] :    
        file_def(cmvs_pertable,table)  


### 15- Have a look at the XIOS config file 'file_def' section

In [21]:
%load land.xml

In [ ]:
<file name="em3hr_CNRM-CM6_Historical ..._r1i1p1f1$TIME_RANGE_PATTERN_FOR_XIOS$_g0"  freq_output="1d" append="true" split_freq="10y" timeseries="exclusive" >
  <variable name="project_id" type="string" > CMIP6/CMIP6 </variable>
  <variable name="institute_id"  type="string" > CNRM </variable>
  <variable name="institute"     type="string" > Centre National de Recherches Météorologiques </variable>
  <variable name="model_id"      type="string" > CNRM-CM6 </variable>
  <variable name="model"         type="string" > CNRM CM6 </variable>
  <variable name="experiment_id" type="string" > historical </variable>
  <variable name="experiment"    type="string" > Historical ... </variable>
  <variable name="frequency"     type="string" > 1d </variable>
  <variable name="realization_index"    type="string" > 1 </variable>
  <variable name="physics_index"        type="string" > 1 </variable>
  <variable name="initialization_index" type="string" > 1 </variable>
  <variable name="forcing_index"        type="string" > 1 </variable>
  <variable name="grid_index"           type="string" > 0 </variable>
  <variable name="parent_experiment_id" type="string" > piControl </variable>
  <variable name="parent_experiment_ripfg" type="string" > r1i0p0f0g0 </variable>
  <variable name="table_id"       type="string" > em3hr </variable>
  <variable name="comment"        type="string" > ... </variable>
  <variable name="history"        type="string" > ... </variable>
  <variable name="references"     type="string" > ... </variable>
  <variable name="title"          type="string" > CNRM-CM6 model output prepared for CMIP6 historical </variable>
  <field field_ref="CF_GrossPrimaryProductivityOfCarbon" operation="average" ts_enabled="true" ts_split_freq="10y">
     <variable name="standard_name" type="string" > GrossPrimaryProductivityOfCarbon </variable>
     <variable name="long_name"     type="string" > Carbon Mass Flux out of Atmosphere due to Gross Primary Production on Land </variable>
     <variable name="description"   type="string" > Gross primary productivity is  ...  </variable>
     <variable name="positive"      type="string" > down </variable>
     <variable name="cell_methods"  type="string" > time: mean area: mean where land </variable>
     <variable name="cell_measures" type="string" > area: areacella </variable>
     </field>
  <field field_ref="CF_PlantRespirationCarbonFlux" operation="average" ts_enabled="true" ts_split_freq="10y">
     <variable name="standard_name" type="string" > PlantRespirationCarbonFlux </variable>
     <variable name="long_name"     type="string" > Carbon Mass Flux into Atmosphere due to Autotrophic (Plant) Respiration on Land </variable>
     <variable name="description"   type="string" > "Respiration carbon" refers to ...  </variable>
     <variable name="positive"      type="string" > up </variable>
     <variable name="cell_methods"  type="string" > time: mean area: mean where land </variable>
     <variable name="cell_measures" type="string" > area: areacella </variable>
     </field>
  <field field_ref="CF_HeterotrophicRespirationCarbonFlux" operation="average" ts_enabled="true" ts_split_freq="10y">
     <variable name="standard_name" type="string" > HeterotrophicRespirationCarbonFlux </variable>
     <variable name="long_name"     type="string" > Carbon Mass Flux into Atmosphere due to Heterotrophic Respiration on Land </variable>
     <variable name="description"   type="string" > "Respiration carbon" refers to ...  </variable>
     <variable name="positive"      type="string" > up </variable>
     <variable name="cell_methods"  type="string" > time: mean area: mean where land </variable>
     <variable name="cell_measures" type="string" > area: areacella </variable>
     </field>
<file/>


### Some checks : list of frequencies for each variable name

In [20]:
vars_freq=dict()
for table in cmvs_pertable :
    for cmv in cmvs_pertable[table] : 
        if cmv.label not in vars_freq : vars_freq[cmv.label]=dict()
        vars_freq[cmv.label][cmv.frequency]=table
#for i in vars_freq : print "% 15s"%i,vars_freq[i]